In [2]:
import tensorflow as tf
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.utils import to_categorical
import numpy as np

# Load CIFAR-10 dataset
(train_images, train_labels), (test_images, test_labels) = cifar10.load_data()
num_classes = 10

# Preprocessing: Normalize pixel values to the range [0, 1]
train_images = train_images.astype('float32') / 255.0
test_images = test_images.astype('float32') / 255.0

# One-hot encode the labels
train_labels = to_categorical(train_labels, num_classes)
test_labels = to_categorical(test_labels, num_classes)

# Basic CNN model
def build_basic_model():
    model = models.Sequential([
        layers.Conv2D(32, (3, 3), activation='relu', padding='same', input_shape=(32, 32, 3)),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.MaxPooling2D((2, 2)),
        layers.Flatten(),
        layers.Dense(256, activation='relu'),
        layers.Dense(num_classes, activation='softmax')
    ])
    return model

# Optimizers
optimizers = ['sgd', 'adam', 'rmsprop']

# Optimizer parameters
sgd_momentum = 0.9
adam_beta_1 = 0.9
adam_beta_2 = 0.999
rmsprop_rho = 0.9

# Regularization
l1_strength = 0.001
l2_strength = 0.001

# Data Preprocessing
def data_preprocessing(images):
    # Apply data preprocessing here if needed (e.g., data augmentation)
    return images

# Define a function to train and evaluate the model
def train_evaluate_model(optimizer, l1_strength=None, l2_strength=None):
    tf.keras.backend.clear_session()
    model = build_basic_model()

    # Add regularization to Conv2D layers
    if l1_strength is not None:
        model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l1(l1_strength)))
    elif l2_strength is not None:
        model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same', kernel_regularizer=regularizers.l2(l2_strength)))

    model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])
    model.fit(data_preprocessing(train_images), train_labels, epochs=10, batch_size=64, verbose=2, validation_split=0.2)

    _, test_accuracy = model.evaluate(data_preprocessing(test_images), test_labels, verbose=0)
    return test_accuracy

# Train and evaluate the model with different optimizers and regularizers
results = {}
for optimizer in optimizers:
    if optimizer == 'sgd':
        optimizer_instance = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=sgd_momentum)
    elif optimizer == 'adam':
        optimizer_instance = tf.keras.optimizers.Adam(learning_rate=0.001, beta_1=adam_beta_1, beta_2=adam_beta_2)
    elif optimizer == 'rmsprop':
        # Use the legacy RMSprop optimizer
        optimizer_instance = tf.keras.optimizers.legacy.RMSprop(learning_rate=0.001, rho=rmsprop_rho)

    results[optimizer] = train_evaluate_model(optimizer_instance)

# Compare the results
print("Accuracy with different optimizers:")
for optimizer, accuracy in results.items():
    print(f"{optimizer}: {accuracy}")

# Compare with and without data preprocessing
no_preprocessing_accuracy = train_evaluate_model(optimizer_instance)
preprocessing_accuracy = train_evaluate_model(optimizer_instance)

print("Accuracy without data preprocessing:", no_preprocessing_accuracy)
print("Accuracy with data preprocessing:", preprocessing_accuracy)


Epoch 1/10
625/625 - 57s - loss: 1.7500 - accuracy: 0.3646 - val_loss: 1.5077 - val_accuracy: 0.4497 - 57s/epoch - 91ms/step
Epoch 2/10
625/625 - 55s - loss: 1.2960 - accuracy: 0.5335 - val_loss: 1.1525 - val_accuracy: 0.5950 - 55s/epoch - 88ms/step
Epoch 3/10
625/625 - 55s - loss: 1.0742 - accuracy: 0.6198 - val_loss: 1.0385 - val_accuracy: 0.6405 - 55s/epoch - 89ms/step
Epoch 4/10
625/625 - 58s - loss: 0.9241 - accuracy: 0.6749 - val_loss: 0.9840 - val_accuracy: 0.6613 - 58s/epoch - 93ms/step
Epoch 5/10
625/625 - 58s - loss: 0.8012 - accuracy: 0.7195 - val_loss: 0.9238 - val_accuracy: 0.6774 - 58s/epoch - 93ms/step
Epoch 6/10
625/625 - 59s - loss: 0.6957 - accuracy: 0.7563 - val_loss: 0.8791 - val_accuracy: 0.7005 - 59s/epoch - 94ms/step
Epoch 7/10
625/625 - 59s - loss: 0.5919 - accuracy: 0.7916 - val_loss: 0.9102 - val_accuracy: 0.6971 - 59s/epoch - 94ms/step
Epoch 8/10
625/625 - 57s - loss: 0.4989 - accuracy: 0.8248 - val_loss: 0.8757 - val_accuracy: 0.7095 - 57s/epoch - 91ms/step
